## Tools for building Henderson's Mixed Model Equations

Use of Gibbs sampler to compute posterior mean of effects 


In [26]:
include("../../PedModule.jl/src/PedModule.jl")
using DataFrames

In [27]:
df = readtable("MTData", separator = ' ')

,y1,y2,y3,trt
1,1.0,2.0,1.2,1
2,1.1,NA,3.1,1
3,0.9,1.9,NA,2
4,1.2,1.7,1.9,2


In [142]:
function Gibbs(A,x,b,nIter;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVarInv = 1.0/A[i,i]
            cMean   = cVarInv*(b[i] - A[:,i]'x)[1,1] + x[i]
            x[i]    = randn()*sqrt(cVarInv) + cMean 
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

function Gibbs(A,x,b)
    n = size(x,1)
    for i=1:n
        cVarInv = 1.0/A[i,i]
        cMean   = cVarInv*(b[i] - A[:,i]'x)[1,1] + x[i]
        x[i]    = randn()*sqrt(cVarInv) + cMean 
    end
end

function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end


type TermStrVal
    str::AbstractString
    value::Float64
end

type TermLvlVal
    level::AbstractString
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}                    # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end
type MME
    modelVec::Array{AbstractString,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{AbstractString,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    R::Array{Float64,2}
    Ai
    mmePos::Int64
end

type ResVar
    R0::Array{Float64,2}
    RiDict::Dict{BitArray{1},Array{Float64,2}}
end   

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getRi(resVar::ResVar,sel::BitArray{1})
    if haskey(resVar.RiDict,sel)
        return resVar.RiDict[sel]
    end
    n = size(resVar.R0,1)
    RZ = zeros(n,n)
    RZ[sel,sel] = inv(resVar.R0[sel,sel])
    resVar.RiDict[sel] = RZ
    return RZ
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec;symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms]
        for (i,trm) = enumerate(modelTerms) 
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,Array(Float64,1,1),R,0,1)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str]) 
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function mkRi(mme::MME,df::DataFrame)
    resVar = ResVar(mme.R,Dict())
    tstMsng = !isna(df[mme.lhsVec[1]])
    for i=2:size(mme.lhsVec,1)
        tstMsng = [tstMsng !isna(df[mme.lhsVec[i]])]
    end
    n    = size(tstMsng,2)
    nObs = size(tstMsng,1)
    ii = Array(Int64,nObs*n^2)
    jj = Array(Int64,nObs*n^2)
    vv = Array(Float64,nObs*n^2)
    pos = 1
    for i=1:size(tstMsng,1)
        sel = reshape(tstMsng[i,:],n)
        Ri  = getRi(resVar,sel)
        for ti=1:n
            tii = (ti-1)*nObs + i
            for tj=1:n
                tjj = (tj-1)*nObs + i
                ii[pos] = tii
                jj[pos] = tjj
                vv[pos] = Ri[ti,tj]
                pos += 1
            end
        end         
    end
    return sparse(ii,jj,vv)
end

function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = convert(Array,df[mme.lhsVec[1]],0.0)
    for i=2:size(mme.lhsVec,1)
        y    = [y; convert(Array,df[mme.lhsVec[i]],0.0)] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = mkRi(mme,df)
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme::MME)
    end   
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ",keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res;string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1  
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

addA (generic function with 1 method)

In [36]:
ped = PedModule.mkPed("sim.ped");

In [71]:
dfGen = readtable("sim.gen", separator = ' ');

In [186]:
Q = convert(Array{Float64,2},dfGen[:,[2:end]]);
α1 = randn(200)
α2 = randn(200)
a1 = Q*α1
a2 = Q*α2;

In [187]:
D = diagm(vec(sqrt(var([a1 a2],1))'))

2x2 Array{Float64,2}:
 7.76017  0.0    
 0.0      7.87793

In [188]:
R = cor([a1 a2])

2x2 Array{Float64,2}:
 1.0       0.127179
 0.127179  1.0     

In [189]:
G0 = D*R*D

2x2 Array{Float64,2}:
 60.2203    7.77497
  7.77497  62.0617 

In [190]:
R0 = diagm(vec(var([a1 a2],1)))
L  = chol(R0)
e  = L*randn(2,size(Q,1))
y = [a1 a2] + e'

4099x2 Array{Float64,2}:
 12.3434   -14.4162  
 10.6911    -8.29442 
  4.9231   -11.1404  
 29.1223     0.781507
 27.8371    -8.36294 
 45.4527   -15.0233  
 35.2718   -17.8208  
 15.4855    -1.32301 
 28.6585    17.1242  
 13.1969   -13.1547  
 13.1458     1.65684 
  8.93899    5.93475 
  2.92263   -9.28481 
  ⋮                  
 30.2266    12.9229  
  8.58454    6.3041  
 11.1911    14.9137  
 15.9977    -2.12932 
 18.3902     8.44056 
 19.0763    12.3756  
 15.0206     8.82024 
 16.5486   -16.7435  
 15.3818   -11.1776  
 29.7432    13.4817  
 17.5011     1.3889  
 25.4999    -4.37301 

In [191]:
df2 = DataFrame(Animal = dfGen[:,1], y1=y[:,1],y2=y[:,2])

,Animal,y1,y2
1,10102,12.343394637329277,-14.416170871641544
2,10103,10.691093256615286,-8.29442494362765
3,10104,4.923097924426148,-11.140381653915187
4,10105,29.122318576146988,0.7815072050102758
5,10106,27.837082855051662,-8.362936636431938
6,10107,45.452685240927224,-15.023310107533716
7,10108,35.271827363069846,-17.82079754523759
8,10109,15.485540228159124,-1.3230145580163193
9,10110,28.658486788152132,17.12422470746872
10,10111,13.196916403376257,-13.154675664946197


In [192]:
models = "y1 = intercept + Animal;
          y2 = intercept + Animal"
R = R0
mme = initMME(models,R)
setAsRandom(mme,"Animal", ped,G0)

In [193]:
getMME(mme,df2)
p = size(mme.mmeLhs,1)

8402

In [194]:
sol = GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)
nothing

1 0.004268438342663815
2 0.0014803809208043964
3 0.0005665458979031832
4 0.00023025007397443114
5 9.755349747812393e-5
6 4.267614868376887e-5
7 1.9349097362797807e-5
8 9.273725500377144e-6
9 4.876099532847077e-6
10 2.9380720386259348e-6
11 2.072297845395653e-6
12 1.6758345069713013e-6
13 1.4853901571982385e-6
14 1.3856457054018197e-6
15 1.3259991987490805e-6
16 1.2842300841377733e-6
17 1.2505875504399187e-6
18 1.2208042188732654e-6
19 1.1930222168967048e-6
20 1.166438863927645e-6
21 1.1407099988567471e-6
22 1.1156877569013898e-6
23 1.0913060920978227e-6
24 1.067531506060218e-6
25 1.0443424055431227e-6
26 1.0217208937806075e-6
27 9.996498381270461e-7


In [183]:
pedTrmVec

2-element Array{AbstractString,1}:
 "1:Animal"
 "2:Animal"

In [200]:
nIter = 1
Profile.clear()
Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
nothing
#[mme.mmeLhs*solGibbs mme.mmeRhs]

S = zeros(Float64,2,2)
for (i,trmi) = enumerate(pedTrmVec)
    pedTrmi  = mme.modelTermDict[trmi]
    startPosi  = pedTrmi.startPos
    endPosi    = startPosi + pedTrmi.nLevels - 1
    for (j,trmj) = enumerate(pedTrmVec)
        pedTrmj  = mme.modelTermDict[trmj]
        startPosj  = pedTrmj.startPos
        endPosj    = startPosj + pedTrmj.nLevels - 1
        S[i,j] = (sol[startPosi:endPosi]'*mme.Ai*sol[startPosj:endPosj])[1,1]
    end
end

ν = 10
P = G0*(ν - 2 - 1)
rand(InverseWishart(ν + 4200, P + S))

2x2 Array{Float64,2}:
 62.3376    8.27413
  8.27413  66.421  

In [196]:
G0

2x2 Array{Float64,2}:
 60.2203    7.77497
  7.77497  62.0617 

In [204]:
function addOne(x)
    x = x + 1
end
x = [1;2]
addOne(x)

2-element Array{Int64,1}:
 2
 3

In [205]:
x

2-element Array{Int64,1}:
 1
 2

In [170]:
function addTwo(x)
    for i in 1:size(x,1)
        x[i] += 2
    end
end

In [206]:
x

2-element Array{Int64,1}:
 1
 2

In [207]:
addTwo(x)

In [208]:
x

2-element Array{Int64,1}:
 3
 4

In [212]:
function addThree(x)
    for i in x
        i = i + 3
    end
    x
end

addThree (generic function with 1 method)

In [209]:
x

2-element Array{Int64,1}:
 3
 4

In [210]:
addThree(x)

2-element Array{Int64,1}:
 3
 4

In [221]:
function addFour(x)
    x[:] += 4
end

addFour (generic function with 1 method)

In [222]:
x

2-element Array{Int64,1}:
 7
 8

In [223]:
addFour(x)

2-element Array{Int64,1}:
 11
 12

In [224]:
x

2-element Array{Int64,1}:
 11
 12